<a href="https://colab.research.google.com/github/sidd-2203/Intrusion-Detection/blob/main/IntrusionDetectionPractise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection System
On NSL-KDD Dataset trying to analyse performance using SVM and learn

In [55]:
import pandas as pd
import numpy as np
import csv
from sklearn import svm
import matplotlib.pyplot as plt

In [56]:
data_file_path='https://raw.githubusercontent.com/sidd-2203/Intrusion-Detection/main/csv_result-KDDTrain%2B.csv'

In [57]:
# Load the CSV file
data=pd.read_csv(data_file_path)

for col_name in data.columns:
  if data[col_name].dtypes=="object":
    unique_cat=len(data[col_name].unique())
    print("{col_names} has following distinct values {values} -> {len}\n".format(col_names=col_name,values=data[col_name].unique(),len=unique_cat))

protocol_type has following distinct values ['tcp' 'udp' 'icmp'] -> 3

service has following distinct values ['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001' 'aol' 'http_2784' 'tftp_u' 'harvest'] -> 70

flag has following distinct values ['SF' 'S0' 'REJ' 'RSTR' 'SH' 'RSTO' 'S1' 'RSTOS0' 'S3' 'S2' 'OTH'] -> 11

class has following distinct values ['normal' 'anomaly'] -> 2



In [58]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           125973 non-null  int64  
 1   duration                     125973 non-null  int64  
 2   protocol_type                125973 non-null  object 
 3   service                      125973 non-null  object 
 4   flag                         125973 non-null  object 
 5   src_bytes                    125973 non-null  int64  
 6   dst_bytes                    125973 non-null  int64  
 7   land                         125973 non-null  int64  
 8   wrong_fragment               125973 non-null  int64  
 9   urgent                       125973 non-null  int64  
 10  hot                          125973 non-null  int64  
 11  num_failed_logins            125973 non-null  int64  
 12  logged_in                    125973 non-null  int64  
 13 

,id,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,1,0,tcp,ftp_data,SF,491,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,2,0,udp,other,SF,146,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,3,0,tcp,private,S0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,4,0,tcp,http,SF,232,8153,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,5,0,tcp,http,SF,199,420,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


Transform categorical features into numbers using LabelEncoder()

In [59]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type','service','flag']

data_categorical_columns=data[categorical_columns]
#data_categorical_columns.head(10)

data_categorical_enc=data_categorical_columns.apply(LabelEncoder().fit_transform)
data_categorical_enc.head(10)


,protocol_type,service,flag
0,1,20,9
1,2,44,9
2,1,49,5
3,1,24,9
4,1,24,9
5,1,49,1
6,1,49,5
7,1,49,5
8,1,51,5
9,1,49,5


Preparing the column names


In [60]:
unique_names=[]
for col_name in data.columns:
    if data[col_name].dtypes == 'object' and col_name in categorical_columns :
        unique_values = data[col_name].unique()
        mod_col_name=col_name +'_'
        unique_names.extend([mod_col_name + x for x in unique_values])
# unique_names

One-Hot-Encoding

In [61]:
encoder=OneHotEncoder(categories='auto')
one_hot_encoded_data=encoder.fit_transform(data_categorical_enc)
df_cat_data=pd.DataFrame(one_hot_encoded_data.toarray(),columns=unique_names)
df_cat_data

,protocol_type_tcp,protocol_type_udp,protocol_type_icmp,service_ftp_data,service_other,service_private,service_http,service_remote_job,service_name,service_netbios_ns,...,flag_S0,flag_REJ,flag_RSTR,flag_SH,flag_RSTO,flag_S1,flag_RSTOS0,flag_S3,flag_S2,flag_OTH
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
125969,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125970,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125971,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [62]:
newdf=data.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)

label_res=newdf['class']
label_res=label_res.replace({'normal':0,'anomaly':1})
newdf['class']=label_res

# newdf.drop('class',axis=1,inplace=True)

In [63]:
Y_df=label_res
X_df=newdf.drop('class')
X_df.head(10)

KeyError: ignored

In [ ]:
colNames=list(X_df)

In [ ]:
from sklearn import preprocessing
scaler1 = preprocessing.StandardScaler().fit(X_df)
X_df=scaler1.transform(X_df)
print(X_df.shape)